## Calculate the ZA transport tensor

Here we take the time and spatial averaged fields, averaged on model Zgrid, and estiamte the corresponding transport tensor. 

In [1]:
import numpy as np
import xarray as xr
#from funcs import *
%matplotlib inline
from matplotlib import pyplot as plt
from matplotlib import colors, ticker, cm
from matplotlib.colors import LogNorm

In [2]:
from xgcm import Grid
from scipy.linalg import pinv, eig, eigh

In [3]:
import sys
sys.path.append('./modules')
import funcs

In [4]:
Model_av = xr.open_dataset('./analysis_data_files/ZA_model_STav_online_Lf_100km.nc')
Trac_av  = xr.open_dataset('./analysis_data_files/ZA_trac_STav_online_Lf_100km.nc')

In [5]:
grid = Grid(Model_av, periodic='X')

In [6]:
list_trac = ['TRAC01', 'TRAC02', 'TRAC03', 'TRAC04', 'TRAC05', 
            'TRAC06', 'TRAC07', 'TRAC08', 'TRAC09', 'TRAC10', 
            'TRAC11', 'TRAC12', 'TRAC13', 'TRAC14', 'TRAC15', 
            'TRAC16', 'TRAC17', 'TRAC18', 'TRAC19', 'TRAC20']

### Generate data for estimating transport tensor

In [9]:
# Make a single dataset with all the gradients 
ds_grads = xr.Dataset()

dx = 5e3 # model resolution 

for var_name in list_trac: 
    
    ds_grads['d'+var_name+'dx'] = grid.interp(grid.diff(Trac_av[var_name], 'X')/dx, 'X')
    ds_grads['d'+var_name+'dy'] = grid.interp(grid.diff(Trac_av[var_name], 'Y', boundary='extend')/dx,
                                              'Y', boundary='extend')
    ds_grads['d'+var_name+'dz'] = grid.interp(grid.diff(Trac_av[var_name], 'Z', boundary='extend')/
                                              grid.diff(Trac_av['Z'], 'Z', boundary='extend'), 
                                              'Z', boundary='extend')

In [8]:
# Make a single dataset with the eddy fluxes (upcp)
#ds_eddy = xr.Dataset()

#for var_name in list_trac:
    
#    ds_eddy['Up'+var_name+'p'] = (grid.interp(Trac_Tav['U'+var_name],'X') - 
#                                  grid.interp(Model_Tav.uVeltave,'X')*Trac_Tav[var_name])
#    ds_eddy['Vp'+var_name+'p'] = (grid.interp(Trac_Tav['V'+var_name], 'Y', boundary='extend') - 
#                                  grid.interp(Model_Tav.vVeltave,'Y', boundary='extend')*Trac_Tav[var_name])
#    ds_eddy['Wp'+var_name+'p'] = (Trac_Tav['W'+var_name] - 
#                                  grid.interp(Model_Tav.wVeltave,'Z', boundary='extend')*Trac_Tav[var_name])
    

In [7]:
[UpCp, VpCp, WpCp] = funcs.get_flux_arrays(Trac_av, list_trac)

In [10]:
[dCdx, dCdy, dCdz]= funcs.get_grad_arrays(ds_grads, list_trac)

In [11]:
UpCp

<xarray.DataArray 'UpCp' (tracer_num: 20, Z: 40, YC: 400, XC: 400)>
array([[[[ 2.64855009e-02,  2.63442714e-02,  2.61911005e-02, ...,
           2.68337000e-02,  2.67304685e-02,  2.66143456e-02],
         [ 2.58810893e-02,  2.57389713e-02,  2.55851895e-02, ...,
           2.62332633e-02,  2.61285137e-02,  2.60110572e-02],
         [ 2.42735408e-02,  2.41367444e-02,  2.39890609e-02, ...,
           2.46142522e-02,  2.45125853e-02,  2.43989471e-02],
         ...,
         [ 2.19934192e-02,  2.18903311e-02,  2.17775628e-02, ...,
           2.22427100e-02,  2.21697073e-02,  2.20865943e-02],
         [ 2.44311616e-02,  2.43104994e-02,  2.41789483e-02, ...,
           2.47251913e-02,  2.46386677e-02,  2.45406255e-02],
         [ 2.59876773e-02,  2.58538108e-02,  2.57082731e-02, ...,
           2.63159182e-02,  2.62189396e-02,  2.61094794e-02]],

        [[ 2.61595212e-02,  2.60185357e-02,  2.58656107e-02, ...,
           2.65070852e-02,  2.64040381e-02,  2.62881294e-02],
         [ 2.56352294e-02,  2.54933946e-02,  2.53399014e-02, ...,
           2.59866863e-02,  2.58821435e-02,  2.57649310e-02],
         [ 2.41057128e-02,  2.39692219e-02,  2.38218531e-02, ...,
           2.44456511e-02,  2.43442059e-02,  2.42308304e-02],
...
          -1.99309504e-03, -1.96394883e-03, -1.93460891e-03],
         [-2.00998457e-03, -1.97806535e-03, -1.94604695e-03, ...,
          -2.10494082e-03, -2.07345467e-03, -2.04179017e-03],
         [-2.03250162e-03, -1.99902942e-03, -1.96545478e-03, ...,
          -2.13219086e-03, -2.09910842e-03, -2.06586905e-03]],

        [[-1.20042171e-03, -1.16697699e-03, -1.13335717e-03, ...,
          -1.30020734e-03, -1.26699544e-03, -1.23374630e-03],
         [-1.00666750e-03, -9.74466093e-04, -9.42109153e-04, ...,
          -1.10296719e-03, -1.07086636e-03, -1.03878323e-03],
         [-7.80968927e-04, -7.50935636e-04, -7.20774755e-04, ...,
          -8.71018507e-04, -8.40954483e-04, -8.10953788e-04],
         ...,
         [-1.48260966e-03, -1.45123666e-03, -1.41969789e-03, ...,
          -1.57569395e-03, -1.54484320e-03, -1.51381246e-03],
         [-1.44252833e-03, -1.40959350e-03, -1.37647800e-03, ...,
          -1.54039916e-03, -1.50791882e-03, -1.47529785e-03],
         [-1.34826545e-03, -1.31458836e-03, -1.28072780e-03, ...,
          -1.44853024e-03, -1.41520752e-03, -1.38179353e-03]]]],
      dtype=float32)
Coordinates:
  * XC          (XC) float32 2.5e+03 7.5e+03 1.25e+04 ... 1.992e+06 1.998e+06
  * YC          (YC) float32 2.5e+03 7.5e+03 1.25e+04 ... 1.992e+06 1.998e+06
  * Z           (Z) float32 -5.0 -15.0 -25.0 ... -2.728e+03 -2.83e+03 -2.934e+03
    rA          (YC, XC) float32 2.5e+07 2.5e+07 2.5e+07 ... 2.5e+07 2.5e+07
    Depth       (YC, XC) float32 0.0 0.0 0.0 ... 2.985e+03 2.985e+03 2.985e+03
    drF         (Z) float32 10.0 10.0 10.0 12.0 14.0 ... 103.0 103.0 103.0 103.0
    PHrefC      (Z) float32 49.05 147.1 245.2 ... 2.676e+04 2.777e+04 2.878e+04
    hFacC       (Z, YC, XC) float32 0.0 0.0 0.0 0.0 0.0 ... 1.0 1.0 1.0 1.0 1.0
    maskC       (Z, YC, XC) int8 0 0 0 0 0 0 0 0 0 0 0 ... 1 1 1 1 1 1 1 1 1 1 1
  * tracer_num  (tracer_num) int64 1 2 3 4 5 6 7 8 9 ... 13 14 15 16 17 18 19 20

In [11]:
# function to estimate the diffusivity tensor
def calc_tensor(uc,vc,wc, cx,cy,cz):
    Aflux = np.array([uc, vc, wc])
    Agrad = np.array([cx, cy, cz])

    if ~(np.isnan(Agrad).any() | np.isnan(Aflux).any()):
        return -(Aflux.dot(pinv(Agrad)))
    else:
        return np.nan*(Aflux.dot(Agrad.T))

In [17]:
%%time
# Calculate the tensor
# This is slow (can take over an hour)

Ktensor_fast = xr.apply_ufunc(calc_tensor, 
                       UpCp.sel(tracer_num=slice(1,19,2)).chunk({'XC':40, 'YC':40, 'Z':10}),
                       VpCp.sel(tracer_num=slice(1,19,2)).chunk({'XC':40, 'YC':40, 'Z':10}),
                       WpCp.sel(tracer_num=slice(1,19,2)).chunk({'XC':40, 'YC':40, 'Z':10}),
                       dCdx.sel(tracer_num=slice(1,19,2)).chunk({'XC':40, 'YC':40, 'Z':10}),
                       dCdy.sel(tracer_num=slice(1,19,2)).chunk({'XC':40, 'YC':40, 'Z':10}),
                       dCdz.sel(tracer_num=slice(1,19,2)).chunk({'XC':40, 'YC':40, 'Z':10}),
                       input_core_dims=[['tracer_num'], ['tracer_num'], ['tracer_num'], ['tracer_num'], ['tracer_num'], ['tracer_num']],
                       vectorize=True, output_core_dims=[['i','j']], dask='parallelized', 
                       output_dtypes=['float32'], output_sizes={'i':3,'j':3})

Ktensor_fast.load(); # need to load because we will take transpose and a

/home/dbalwada/.conda/envs/pangeo/lib/python3.7/site-packages/ipykernel/__main__.py:13: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.


CPU times: user 1h 8min 47s, sys: 28min, total: 1h 36min 48s
Wall time: 54min 14s


<xarray.DataArray (Z: 40, YC: 400, XC: 400, i: 3, j: 3)>
array([[[[[            nan,             nan,             nan],
          [            nan,             nan,             nan],
          [            nan,             nan,             nan]],

         [[            nan,             nan,             nan],
          [            nan,             nan,             nan],
          [            nan,             nan,             nan]],

         [[            nan,             nan,             nan],
          [            nan,             nan,             nan],
          [            nan,             nan,             nan]],

         ...,

         [[            nan,             nan,             nan],
          [            nan,             nan,             nan],
          [            nan,             nan,             nan]],

         [[            nan,             nan,             nan],
          [            nan,             nan,             nan],
...
          [ 3.94610576e-02,  8.80180746e-02,  1.35392032e-03]],

         [[ 1.17063809e+04,  1.74213757e+03, -1.82171011e+00],
          [-5.56428772e+02, -1.00318153e+02,  1.26567930e-01],
          [ 7.15919808e-02,  8.83120075e-02,  1.36450108e-03]],

         ...,

         [[ 1.03401465e+04,  1.79935913e+03, -2.13230205e+00],
          [-4.36664368e+02, -9.65018234e+01,  1.93306893e-01],
          [-7.94607773e-02,  8.64972770e-02,  1.30229327e-03]],

         [[ 1.06392070e+04,  1.78974622e+03, -2.05418158e+00],
          [-4.60532684e+02, -9.73182678e+01,  1.79192007e-01],
          [-5.36595173e-02,  8.69364738e-02,  1.31577312e-03]],

         [[ 1.09397939e+04,  1.77908179e+03, -1.98483360e+00],
          [-4.85909058e+02, -9.81245193e+01,  1.65324867e-01],
          [-2.45173834e-02,  8.73349756e-02,  1.32918707e-03]]]]],
      dtype=float32)
Coordinates:
  * XC       (XC) float32 2.5e+03 7.5e+03 1.25e+04 ... 1.992e+06 1.998e+06
  * YC       (YC) float32 2.5e+03 7.5e+03 1.25e+04 ... 1.992e+06 1.998e+06
  * Z        (Z) float32 -5.0 -15.0 -25.0 ... -2.728e+03 -2.83e+03 -2.934e+03
    rA       (YC, XC) float32 2.5e+07 2.5e+07 2.5e+07 ... 2.5e+07 2.5e+07
    Depth    (YC, XC) float32 0.0 0.0 0.0 0.0 ... 2.985e+03 2.985e+03 2.985e+03
    drF      (Z) float32 10.0 10.0 10.0 12.0 14.0 ... 103.0 103.0 103.0 103.0
    PHrefC   (Z) float32 49.05 147.1 245.2 ... 2.676e+04 2.777e+04 2.878e+04
    hFacC    (Z, YC, XC) float32 0.0 0.0 0.0 0.0 0.0 0.0 ... 1.0 1.0 1.0 1.0 1.0
    maskC    (Z, YC, XC) int8 0 0 0 0 0 0 0 0 0 0 0 0 ... 1 1 1 1 1 1 1 1 1 1 1
Dimensions without coordinates: i, j

In [18]:
%%time
# Calculate the tensor
# This is slow (can take over an hour)

Ktensor_slow = xr.apply_ufunc(calc_tensor, 
                       UpCp.sel(tracer_num=slice(2,20,2)).chunk({'XC':40, 'YC':40, 'Z':10}),
                       VpCp.sel(tracer_num=slice(2,20,2)).chunk({'XC':40, 'YC':40, 'Z':10}),
                       WpCp.sel(tracer_num=slice(2,20,2)).chunk({'XC':40, 'YC':40, 'Z':10}),
                       dCdx.sel(tracer_num=slice(2,20,2)).chunk({'XC':40, 'YC':40, 'Z':10}),
                       dCdy.sel(tracer_num=slice(2,20,2)).chunk({'XC':40, 'YC':40, 'Z':10}),
                       dCdz.sel(tracer_num=slice(2,20,2)).chunk({'XC':40, 'YC':40, 'Z':10}),
                       input_core_dims=[['tracer_num'], ['tracer_num'], ['tracer_num'], ['tracer_num'], ['tracer_num'], ['tracer_num']],
                       vectorize=True, output_core_dims=[['i','j']], dask='parallelized', 
                       output_dtypes=['float32'], output_sizes={'i':3,'j':3})

Ktensor_slow.load(); # need to load because we will take transpose and a

/home/dbalwada/.conda/envs/pangeo/lib/python3.7/site-packages/ipykernel/__main__.py:13: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.


CPU times: user 1h 7min 57s, sys: 27min 49s, total: 1h 35min 47s
Wall time: 54min 2s


<xarray.DataArray (Z: 40, YC: 400, XC: 400, i: 3, j: 3)>
array([[[[[            nan,             nan,             nan],
          [            nan,             nan,             nan],
          [            nan,             nan,             nan]],

         [[            nan,             nan,             nan],
          [            nan,             nan,             nan],
          [            nan,             nan,             nan]],

         [[            nan,             nan,             nan],
          [            nan,             nan,             nan],
          [            nan,             nan,             nan]],

         ...,

         [[            nan,             nan,             nan],
          [            nan,             nan,             nan],
          [            nan,             nan,             nan]],

         [[            nan,             nan,             nan],
          [            nan,             nan,             nan],
...
          [ 6.98047936e-01,  6.30296990e-02,  1.26520358e-03]],

         [[ 1.74231035e+04,  1.57377332e+03, -3.38537073e+00],
          [-8.76112793e+02, -1.03265793e+02,  1.63861424e-01],
          [ 7.37077415e-01,  6.34517670e-02,  1.26914529e-03]],

         ...,

         [[ 1.57762100e+04,  1.61878113e+03, -3.27790594e+00],
          [-7.01549255e+02, -9.93734436e+01,  2.28238583e-01],
          [ 5.10140121e-01,  6.12538382e-02,  1.23007153e-03]],

         [[ 1.61928164e+04,  1.61062683e+03, -3.27215314e+00],
          [-7.38450989e+02, -1.00182442e+02,  2.13577151e-01],
          [ 5.56468666e-01,  6.16946369e-02,  1.24077010e-03]],

         [[ 1.65906152e+04,  1.60202148e+03, -3.28310585e+00],
          [-7.76647034e+02, -1.00998474e+02,  1.99677765e-01],
          [ 6.05069518e-01,  6.21425323e-02,  1.25057483e-03]]]]],
      dtype=float32)
Coordinates:
  * XC       (XC) float32 2.5e+03 7.5e+03 1.25e+04 ... 1.992e+06 1.998e+06
  * YC       (YC) float32 2.5e+03 7.5e+03 1.25e+04 ... 1.992e+06 1.998e+06
  * Z        (Z) float32 -5.0 -15.0 -25.0 ... -2.728e+03 -2.83e+03 -2.934e+03
    rA       (YC, XC) float32 2.5e+07 2.5e+07 2.5e+07 ... 2.5e+07 2.5e+07
    Depth    (YC, XC) float32 0.0 0.0 0.0 0.0 ... 2.985e+03 2.985e+03 2.985e+03
    drF      (Z) float32 10.0 10.0 10.0 12.0 14.0 ... 103.0 103.0 103.0 103.0
    PHrefC   (Z) float32 49.05 147.1 245.2 ... 2.676e+04 2.777e+04 2.878e+04
    hFacC    (Z, YC, XC) float32 0.0 0.0 0.0 0.0 0.0 0.0 ... 1.0 1.0 1.0 1.0 1.0
    maskC    (Z, YC, XC) int8 0 0 0 0 0 0 0 0 0 0 0 0 ... 1 1 1 1 1 1 1 1 1 1 1
Dimensions without coordinates: i, j

In [19]:
lam1 = 1/31104000.
lam2 = 1/186624000.

Ktensor_corr = (lam2*Ktensor_fast - lam1*Ktensor_slow)/(lam2 - lam1)

In [20]:
Ktensor_corrT = Ktensor_corr.transpose('Z','YC','XC','j','i')

STcorr = 0.5*(Ktensor_corr.data + Ktensor_corrT.data)
ATcorr = 0.5*(Ktensor_corr.data - Ktensor_corrT.data)

STcorr= xr.DataArray(STcorr, coords=Ktensor_corr.coords, dims=Ktensor_corr.dims)
ATcorr = xr.DataArray(ATcorr, coords=Ktensor_corr.coords, dims=Ktensor_corr.dims)

In [21]:
# make sure the eigen values are arranged by magnitude (instead of the default arrangement)

def eigen(A):
    if np.isnan(A).any(): A = np.nan_to_num(A)
    eigenValues, eigenVectors = eigh(A)
    idx = np.argsort(np.abs(eigenValues))
    eigenValues = eigenValues[idx]
    eigenVectors = eigenVectors[:,idx]
    return (eigenValues, eigenVectors)

In [22]:
# calculate the eigenvalues and eigenvectors of symmetric part
eigvalsSTcorr, eigvecsSTcorr = xr.apply_ufunc(eigen, STcorr, input_core_dims=[['i','j']],
                                    vectorize=True, output_core_dims=[['ii'], ['k','ii']])

In [23]:
diff_tensor = xr.Dataset()
diff_tensor['Kfast'] = Ktensor_fast
diff_tensor['Kslow'] = Ktensor_slow
diff_tensor['Kcorr'] = Ktensor_corr
diff_tensor['STcorr'] = STcorr
diff_tensor['ATcorr'] = ATcorr
diff_tensor['eigvalsSTcorr'] = eigvalsSTcorr
diff_tensor['eigvecsSTcorr'] = eigvecsSTcorr

In [25]:
diff_tensor.to_netcdf('analysis_data_files/diff_tensor_ZA_100km.nc')